### 광석과 돌 분류를 위한 딥러닝

#### K-겹 교차 검증 (K-fold Cross Validation)

In [1]:
from keras.models import Sequential, load_model
from keras.layers.core import Dense
from sklearn.preprocessing import LabelEncoder

import pandas as pd
import numpy as np
import tensorflow as tf

Using TensorFlow backend.


In [2]:
seed = 0
np.random.seed(seed)
tf.random.set_seed(3)

In [3]:
df = pd.read_csv('dataset/sonar.csv', header=None)
df

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0.0187,0.0346,0.0168,0.0177,0.0393,0.1630,0.2028,0.1694,0.2328,0.2684,...,0.0116,0.0098,0.0199,0.0033,0.0101,0.0065,0.0115,0.0193,0.0157,M
204,0.0323,0.0101,0.0298,0.0564,0.0760,0.0958,0.0990,0.1018,0.1030,0.2154,...,0.0061,0.0093,0.0135,0.0063,0.0063,0.0034,0.0032,0.0062,0.0067,M
205,0.0522,0.0437,0.0180,0.0292,0.0351,0.1171,0.1257,0.1178,0.1258,0.2529,...,0.0160,0.0029,0.0051,0.0062,0.0089,0.0140,0.0138,0.0077,0.0031,M
206,0.0303,0.0353,0.0490,0.0608,0.0167,0.1354,0.1465,0.1123,0.1945,0.2354,...,0.0086,0.0046,0.0126,0.0036,0.0035,0.0034,0.0079,0.0036,0.0048,M


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208 entries, 0 to 207
Data columns (total 61 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       208 non-null    float64
 1   1       208 non-null    float64
 2   2       208 non-null    float64
 3   3       208 non-null    float64
 4   4       208 non-null    float64
 5   5       208 non-null    float64
 6   6       208 non-null    float64
 7   7       208 non-null    float64
 8   8       208 non-null    float64
 9   9       208 non-null    float64
 10  10      208 non-null    float64
 11  11      208 non-null    float64
 12  12      208 non-null    float64
 13  13      208 non-null    float64
 14  14      208 non-null    float64
 15  15      208 non-null    float64
 16  16      208 non-null    float64
 17  17      208 non-null    float64
 18  18      208 non-null    float64
 19  19      208 non-null    float64
 20  20      208 non-null    float64
 21  21      208 non-null    float64
 22  22

In [5]:
dataset = df.values
dataset

array([[0.02, 0.0371, 0.0428, ..., 0.009, 0.0032, 'R'],
       [0.0453, 0.0523, 0.0843, ..., 0.0052, 0.0044, 'R'],
       [0.0262, 0.0582, 0.1099, ..., 0.0095, 0.0078, 'R'],
       ...,
       [0.0522, 0.0437, 0.018, ..., 0.0077, 0.0031, 'M'],
       [0.0303, 0.0353, 0.049, ..., 0.0036, 0.0048, 'M'],
       [0.026, 0.0363, 0.0136, ..., 0.0061, 0.0115, 'M']], dtype=object)

In [6]:
X = dataset[:,0:60]
Y_obj = dataset[:,60]
print(Y_obj)

['R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R'
 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R'
 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R'
 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R'
 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R'
 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M'
 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M'
 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M'
 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M'
 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M'
 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M'
 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M']


In [7]:
# Y 클래스 ONE-HOT 인코딩
e = LabelEncoder()
e.fit(Y_obj)
Y = e.transform(Y_obj)
Y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

#### K-fold Cross Validation 사용  


##### sklearn에서 제공하는 StratifiedKFold 함수를 사용  
##### 옵션 중에서 n_splits 부분 : K-겹에서 K에 해당하는 값,  5로 설정할 경우 5개의 그룹으로 데이터셋을 나눈다는 뜻

In [8]:
from sklearn.model_selection import StratifiedKFold

In [9]:
n_fold = 5  # 5개의 그룹으로 데이터셋 나눔
skf = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)  # shuffle : 데이터셋 섞기
skf

StratifiedKFold(n_splits=5, random_state=0, shuffle=True)

In [10]:
accuracy = []
step = 0

for train, test in skf.split(X, Y):
    step += 1
    print("* K : %d 번째 iteration" %step)
    
    model = Sequential()
    model.add(Dense(24, input_dim=60, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
#    print(train)
#    print(test)
    
    model.fit(X[train], Y[train], epochs=10, batch_size=5, verbose=1)
    
    k_loss, k_accuracy = model.evaluate(x=X[test], y=Y[test], verbose=1)
    print("Evaluate with test dataset : Loss = {:.4f}".format(k_loss))
    print("Evaluate with test dataset : Accuracy = {:.4f}".format(k_accuracy))
    
    accuracy.append(k_accuracy)

* K : 1 번째 iteration
Epoch 1/10
166/166 [==============================] - 0s 3ms/step - loss: 0.6873 - accuracy: 0.5361
Epoch 2/10
166/166 [==============================] - 0s 430us/step - loss: 0.6489 - accuracy: 0.6687
Epoch 3/10
166/166 [==============================] - 0s 457us/step - loss: 0.6123 - accuracy: 0.7651
Epoch 4/10
166/166 [==============================] - 0s 312us/step - loss: 0.5800 - accuracy: 0.7711
Epoch 5/10
166/166 [==============================] - 0s 481us/step - loss: 0.5513 - accuracy: 0.7530
Epoch 6/10
166/166 [==============================] - 0s 397us/step - loss: 0.5219 - accuracy: 0.7771
Epoch 7/10
166/166 [==============================] - 0s 625us/step - loss: 0.4952 - accuracy: 0.7771
Epoch 8/10
166/166 [==============================] - 0s 595us/step - loss: 0.4825 - accuracy: 0.7711
Epoch 9/10
166/166 [==============================] - 0s 472us/step - loss: 0.4490 - accuracy: 0.7952
Epoch 10/10
42/42 [==============================] - 0s 558us/s

In [11]:
print("K-fold Cross Validation을 사용한 딥러닝")
print("K 값 :  %.f " % n_fold)
print("Accuracy : ", accuracy) # accuracy 리스트 값 출력 
print("Accuracy의 K 번 iteration 마다 정확도를 의미한다.")

K-fold Cross Validation을 사용한 딥러닝
K 값 :  5 
Accuracy :  [0.761904776096344, 0.761904776096344, 0.6666666865348816, 0.707317054271698, 0.707317054271698]
Accuracy의 K 번 iteration 마다 정확도를 의미한다.


In [12]:
a = np.array(accuracy)
print(type(a)) 
print(a.mean()) 

<class 'numpy.ndarray'>
0.7210220694541931
